In [1]:
!pip install pyomo

You should consider upgrading via the '/Users/fpasten/Uni/CINF105/sol1/venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from pyomo.environ import *

# Problema 1

In [5]:
# Creando modelo base
problem_1 = ConcreteModel()

# set I 1,2
problem_1.I = RangeSet(1, 2)

# set J 1,2
problem_1.J = RangeSet(1, 2)

# set K 1,2,3
problem_1.K = RangeSet(1, 3)

# flight cost c{I,J} dictionary
flight_cost = { (1,1): 120000, (1,2): 80000, (2,1): 90000, (2,2): 60000}

# param c{I,J}
problem_1.c = Param(problem_1.I, problem_1.J, initialize=flight_cost)

# capacity a{I,K} dictionary
capacity = { (1,1): 20, (1,2): 4, (1,3): 3, (2,1): 15, (2,2): 1, (2,3): 2}

# param a{I,K}
problem_1.a = Param(problem_1.I, problem_1.K, initialize=capacity)

# demand demA{K} dictionary
demand = { 1: 350, 2: 120, 3: 90}

# param demA{K}
problem_1.demA = Param(problem_1.K, initialize=demand)

# param demB
problem_1.demB = Param(initialize=2)

# airport capacity airC{K} dictionary
airport_capacity = { 1: 2000, 2: 150, 3: 230}

# param airC{K}
problem_1.airC = Param(problem_1.K, initialize=airport_capacity)

# var x{I,J} >= 0 integer
problem_1.x = Var(problem_1.I, problem_1.J, domain=NonNegativeIntegers)

# define objective function z sum{i in I, j in J} c[i,j] * x[i,j]
problem_1.z = Objective(expr=sum(problem_1.c[i,j] * problem_1.x[i, j] for i in problem_1.I for j in problem_1.J), sense=minimize)

# define constraint sum{i in I} a[i,k] * x[i,1] >= demA[k] for k in K
problem_1.r1 = ConstraintList()
for k in problem_1.K:
    problem_1.r1.add(sum(problem_1.a[i,k] * problem_1.x[i,1] for i in problem_1.I) >= problem_1.demA[k])

# define constraint sum{i in I} a[i,k] * x[i,2] >= demB 
problem_1.r2 = Constraint(expr=sum(problem_1.a[i,k] * problem_1.x[i,2] for i in problem_1.I) >= problem_1.demB)

# define constraint sum{i in I} x[i,2] * a[i,2] * 1/120 + sum{i in I} x[i,2] * a[i,3] * 1/90 <= 1
problem_1.r3 = Constraint(expr=sum(problem_1.x[i,2] * problem_1.a[i,2] * 1/120 for i in problem_1.I) + sum(problem_1.x[i,2] * problem_1.a[i,3] * 1/90 for i in problem_1.I) <= 1)

# define constraint sum{j in J} x[1,j] * 1/120 + sum{j in J} x[2,j] * 1/85 <= 1
problem_1.r4 = Constraint(expr=sum(problem_1.x[1,j] * 1/120 for j in problem_1.J) + sum(problem_1.x[2,j] * 1/85 for j in problem_1.J) <= 1)




# Usando gurobi como solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_1)

# Imprimiendo resultados
print('Objective function value: ', problem_1.z())
for i in problem_1.I:
    for j in problem_1.J:
        print('x[',i,',',j,'] = ', problem_1.x[i,j]())
print('z = ', value(problem_1.z))

Objective function value:  3660000.0
x[ 1 , 1 ] =  30.0
x[ 1 , 2 ] =  0.0
x[ 2 , 1 ] =  0.0
x[ 2 , 2 ] =  1.0
z =  3660000.0


In [12]:
# Creando modelo base
problem_1 = ConcreteModel()

# Definiendo variables
problem_1.x1 = Var(domain=NonNegativeIntegers)
problem_1.x2 = Var(domain=NonNegativeIntegers)
problem_1.x3 = Var(domain=NonNegativeIntegers)

# Definiendo la función objetivo
problem_1.z = Objective(expr=70 * problem_1.x1 + 15 * problem_1.x2 + 45 * problem_1.x3, sense=maximize)

# Estableciendo restricciones
problem_1.r1 = Constraint(expr=18 * problem_1.x1 + 2 * problem_1.x2 + 4 * problem_1.x3 <= 54)
problem_1.r2 = Constraint(expr=9 * problem_1.x1 + 2 * problem_1.x2 + 4 * problem_1.x3 <= 30)
problem_1.r3 = Constraint(expr=2 * problem_1.x1 + 1 * problem_1.x2 + 0 * problem_1.x3 <= 30)

# Usando gurobi como solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_1)

# Imprimiendo resultados
print('Solución optima:')
print('x1 =', value(problem_1.x1))
print('x2 =', value(problem_1.x2))
print('x3 =', value(problem_1.x3))
print('Valor óptimo:', value(problem_1.z))

Solución optima:
x1 = 0.0
x2 = 1.0
x3 = 7.0
Valor óptimo: 330.0


# Problema 2

In [9]:
problem_2 = ConcreteModel()

# set I 1,2,3
problem_2.I = RangeSet(1, 3)

# set J 1,2
problem_2.J = RangeSet(1, 2)

# loss dictionary for D{I}
loss = { 1: 62, 2: 1, 3: 30}

# param D{I}
problem_2.D = Param(problem_2.I, initialize=loss)

# cost dictionary for C{J}
cost = { 1: 40, 2: 30}

# param C{J}
problem_2.C = Param(problem_2.J, initialize=cost)

# param B
problem_2.B = Param(initialize=350)

# amount of pieces per cut A{I,J} dictionary
amount = { (1,1): 2, (1,2): 0, (2,1): 1, (2,2): 2, (3,1): 0, (3,2): 3}

# param A{I,J}
problem_2.A = Param(problem_2.I, problem_2.J, initialize=amount)

# param Wcost
problem_2.Wcost = Param(initialize=1)

# var Inv{J} >= 0 integer
problem_2.Inv = Var(problem_2.J, domain=NonNegativeIntegers)

# var x{I} >= 0 integer
problem_2.x = Var(problem_2.I, domain=NonNegativeIntegers)

# define objective function z sum{i in I} D[i] * x[i] * Wcost + sum{j in J} C[j] * Inv[j]
problem_2.z = Objective(expr=sum(problem_2.D[i] * problem_2.x[i] * problem_2.Wcost for i in problem_2.I) + sum(problem_2.C[j] * problem_2.Inv[j] for j in problem_2.J), sense=minimize)

# define constraint sum{i in I} A[i,j] * x[i] >= B for j in J
problem_2.r1 = ConstraintList()
for j in problem_2.J:
    problem_2.r1.add(sum(problem_2.A[i,j] * problem_2.x[i] for i in problem_2.I) >= problem_2.B)

# define constraint Inv[j] = sum{i in I} A[i,j] * x[i] - B for j in J
problem_2.r2 = ConstraintList()
for j in problem_2.J:
    problem_2.r2.add(problem_2.Inv[j] == sum(problem_2.A[i,j] * problem_2.x[i] for i in problem_2.I) - problem_2.B)

# Usando gurobi como solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_2)

# Imprimiendo resultados
print('Objective function value: ', problem_2.z())
for i in problem_2.I:
    print('x[',i,'] = ', problem_2.x[i]())

Objective function value:  5630.0
x[ 1 ] =  87.0
x[ 2 ] =  176.0
x[ 3 ] =  -0.0


In [18]:
problem_2 = ConcreteModel()

problem_2.x1 = Var(domain=NonNegativeIntegers)
problem_2.x2 = Var(domain=NonNegativeIntegers)

problem_2.z = Objective(expr=200 * problem_2.x1 + 120 * problem_2.x2, sense=maximize)

problem_2.r1 = Constraint(expr=10 * problem_2.x1 + 12 * problem_2.x2 <= 3300)
problem_2.r2 = Constraint(expr=30 * problem_2.x1 + 6 * problem_2.x2 <= 6000)
problem_2.r3 = Constraint(expr=problem_2.x1 + problem_2.x2 >= 100)
problem_2.r4 = Constraint(expr=problem_2.x2 >= 0.25 * problem_2.x1)
problem_2.r5 = Constraint(expr=problem_2.x2 <= problem_2.x1 + 150)

solver = SolverFactory('gurobi')
results = solver.solve(problem_2)

print('Solución optima:')
print('x1 =', value(problem_2.x1))
print('x2 =', value(problem_2.x2))
print('Valor óptimo:', value(problem_2.z))

Solución optima:
x1 = 0.0
x2 = 100.0
Valor óptimo: 12000.0


# Problema 3

In [21]:
problem_3 = ConcreteModel()

# set T 1..4
problem_3.T = RangeSet(1, 4)

# demand dictionary for dem{T}
demand = { 1: 2800, 2: 2200, 3: 3200, 4: 2500}

# param dem{T}
problem_3.dem = Param(problem_3.T, initialize=demand)

# param h
problem_3.h = Param(initialize=2)

# param ce
problem_3.ce = Param(initialize=10)

# param cap
problem_3.cap = Param(initialize=2700)

# param capE
problem_3.capE = Param(initialize=300)

# var x{T} >= 0 integer
problem_3.x = Var(problem_3.T, domain=NonNegativeIntegers)

# var e{T} >= 0 integer
problem_3.e = Var(problem_3.T, domain=NonNegativeIntegers)

# var I{T union {0}} >= 0 integer
problem_3.I = Var(problem_3.T.union({0}), domain=NonNegativeIntegers)
#problem_3.I.add(0)
# initialize I[0] = 0
# define objective function z sum{t in T}(e[t] * ce + I[t] * h)
problem_3.z = Objective(expr=sum(problem_3.e[t] * problem_3.ce + problem_3.I[t] * problem_3.h for t in problem_3.T), sense=minimize)

# define constraint x[t] <= cap for t in T
problem_3.r1 = ConstraintList()
for t in problem_3.T:
    problem_3.r1.add(problem_3.x[t] <= problem_3.cap)

# define constraint e[t] <= capE for t in T
problem_3.r2 = ConstraintList()
for t in problem_3.T:
    problem_3.r2.add(problem_3.e[t] <= problem_3.capE)

# define constraint  I[t-1] + x[t] + e[t] = I[t] + dem[t] for t in T
problem_3.r3 = ConstraintList()
for t in problem_3.T:
    problem_3.r3.add(problem_3.I[t-1] + problem_3.x[t] + problem_3.e[t] == problem_3.I[t] + problem_3.dem[t])

# define constraint I[0] = 0
problem_3.r4 = Constraint(expr=problem_3.I[0] == 0)

# define constraint I[4] = 0
problem_3.r5 = Constraint(expr=problem_3.I[4] == 0)

# Usando gurobi como solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_3)

# Imprimiendo resultados
print('Objective function value: ', problem_3.z())
for t in problem_3.T:
    print('x[',t,'] = ', problem_3.x[t]())
    print('e[',t,'] = ', problem_3.e[t]())
    print('I[',t,'] = ', problem_3.I[t]())




Objective function value:  2000.0
x[ 1 ] =  2700.0
e[ 1 ] =  100.0
I[ 1 ] =  0.0
x[ 2 ] =  2700.0
e[ 2 ] =  0.0
I[ 2 ] =  500.0
x[ 3 ] =  2700.0
e[ 3 ] =  0.0
I[ 3 ] =  -0.0
x[ 4 ] =  2500.0
e[ 4 ] =  -0.0
I[ 4 ] =  0.0


## Variación propuesta 

Suponiendo que existe tecnología para pescar el tipo de pescado deseado con 

# Problema 4

In [24]:
problem_4 = ConcreteModel()

# set I 1..4
problem_4.I = RangeSet(1, 4)

# set J 1..3
problem_4.J = RangeSet(1, 3)

# utility dictionary for u{I}
utility = { 1: 250, 2: 280, 3: 500, 4: 360}

# param u{I}
problem_4.u = Param(problem_4.I, initialize=utility)

# max weight capacity dictionary for P{J}
max_weight_capacity = { 1: 16, 2: 20, 3: 14}

# param P{J}
problem_4.P = Param(problem_4.J, initialize=max_weight_capacity)

# max volume capacity dictionary for V{J}
max_volume_capacity = { 1: 200, 2: 250, 3: 150}

# param V{J}
problem_4.V = Param(problem_4.J, initialize=max_volume_capacity)

# load qty dictionary for C{I}
load_qty = { 1: 20, 2: 15, 3: 8, 4: 10}

# param C{I}
problem_4.C = Param(problem_4.I, initialize=load_qty)

# max volume of type dictionary for Vol{I}
max_volume_of_type = { 1: 1, 2: 2, 3: 10, 4: 6}

# param Vol{I}
problem_4.Vol = Param(problem_4.I, initialize=max_volume_of_type)

# var x{I,J} >= 0 integer
problem_4.x = Var(problem_4.I, problem_4.J, domain=NonNegativeIntegers)

# var G >= 0
problem_4.G = Var(domain=NonNegativeReals)

# define objective function z = sum{i in I, j in J} u[i] * x[i,j]
problem_4.z = Objective(expr=sum(problem_4.u[i] * problem_4.x[i,j] for i in problem_4.I for j in problem_4.J), sense=maximize)

# define constraint sum{i in I} x[i,j] <= P[j] for j in J
problem_4.r1 = ConstraintList()
for j in problem_4.J:
    problem_4.r1.add(sum(problem_4.x[i,j] for i in problem_4.I) <= problem_4.P[j])

# define constraint sum{i in I} Vol[i] * x[i,j] <= V[j] for j in J
problem_4.r2 = ConstraintList()
for j in problem_4.J:
    problem_4.r2.add(sum(problem_4.Vol[i] * problem_4.x[i,j] for i in problem_4.I) <= problem_4.V[j])

# define constraint sum{j in J} x[i,j] <= C[i] for i in I
problem_4.r3 = ConstraintList()
for i in problem_4.I:
    problem_4.r3.add(sum(problem_4.x[i,j] for j in problem_4.J) <= problem_4.C[i])

# define constraint sum{i in I} x[i,j]/P[j] == G for j in J
problem_4.r4 = ConstraintList()
for j in problem_4.J:
    problem_4.r4.add(sum(problem_4.x[i,j]/problem_4.P[j] for i in problem_4.I) == problem_4.G)

# Usando gurobi como solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_4)

# Imprimiendo resultados
print('Objective function value: ', problem_4.z())
for i in problem_4.I:
    for j in problem_4.J:
        print('x[',i,',',j,'] = ', problem_4.x[i,j]())





Objective function value:  16050.0
x[ 1 , 1 ] =  -0.0
x[ 1 , 2 ] =  3.0
x[ 1 , 3 ] =  14.0
x[ 2 , 1 ] =  6.0
x[ 2 , 2 ] =  9.0
x[ 2 , 3 ] =  -0.0
x[ 3 , 1 ] =  -0.0
x[ 3 , 2 ] =  8.0
x[ 3 , 3 ] =  0.0
x[ 4 , 1 ] =  10.0
x[ 4 , 2 ] =  -0.0
x[ 4 , 3 ] =  -0.0


# Problema 5

In [9]:
problem_5 = ConcreteModel()

# set I 1..3
problem_5.I = RangeSet(1, 3)

# set J 1..3
problem_5.J = RangeSet(1, 3)

# set K 1..3
problem_5.K = RangeSet(1, 3)

# composition dictionary for PC{J,K}
composition = { (1,1): 80, (1,2): 10, (1,3): 5, (2,1): 45, (2,2): 30, (2,3): 20, (3,1): 30, (3,2): 40, (3,3): 25}

# param PC{J,K}
problem_5.PC = Param(problem_5.J, problem_5.K, initialize=composition)

# required composition in gas dictionary for PG{I,K}
required_composition_in_gas = { (1,1): 60, (1,2): 25, (1,3): 10, (2,1): 30, (2,2): 20, (2,3): 15, (3,1): 40, (3,2): 35, (3,3): 20}

# param PG{I,K}
problem_5.PG = Param(problem_5.I, problem_5.K, initialize=required_composition_in_gas)

# cost per barrel of crude dictionary for Cost{j}
cost_per_barrel_of_crude = { 1: 250, 2: 120, 3: 200}

# param Cost{j}
problem_5.CostoC = Param(problem_5.J, initialize=cost_per_barrel_of_crude)

# param budget
problem_5.Presu = Param(initialize=50000)

# available crude dictionary for type 2 and 3 
available_crude = { 2: 300, 3: 700}

# param available_crude
problem_5.available_crude = Param(RangeSet(2,3), initialize=available_crude)

# param barrels of crude type A to buy
problem_5.BA = Param(initialize=10)

# demand dictionary for type 1 and 2
demand = { 1: 60, 2: 70}

# param demand
problem_5.DEM = Param(RangeSet(1,2), initialize=demand)

# Define the variables
problem_5.X = Var(problem_5.I, problem_5.J, within=NonNegativeReals)

# Define the objective function
problem_5.z = Objective(expr=sum(problem_5.X[3, j] for j in problem_5.J), sense=maximize)

# Define the constraints
problem_5.r1 = ConstraintList()
for k in [1, 3]:
    problem_5.r1.add(sum(problem_5.PC[j, k] * problem_5.X[1, j] for j in problem_5.J) >= problem_5.PG[1, k] * sum(problem_5.X[1, j] for j in problem_5.J))

problem_5.r2 = Constraint(expr=sum(problem_5.PC[j, 2] * problem_5.X[1, j] for j in problem_5.J) <= problem_5.PG[1, 2] * sum(problem_5.X[1, j] for j in problem_5.J))

problem_5.r3 = ConstraintList()
for k in [2, 3]:
    problem_5.r3.add(sum(problem_5.PC[j, k] * problem_5.X[2, j] for j in problem_5.J) <= problem_5.PG[2, k] * sum(problem_5.X[2, j] for j in problem_5.J))

problem_5.r4 = Constraint(expr=sum(problem_5.PC[j, 1] * problem_5.X[2, j] for j in problem_5.J) >= problem_5.PG[2, 1] * sum(problem_5.X[2, j] for j in problem_5.J))

problem_5.r5 = ConstraintList()
for k in [2, 3]:
    problem_5.r5.add(sum(problem_5.PC[j, k] * problem_5.X[3, j] for j in problem_5.J) >= problem_5.PG[3, k] * sum(problem_5.X[3, j] for j in problem_5.J))

problem_5.r6 = Constraint(expr=sum(problem_5.PC[j, 1] * problem_5.X[3, j] for j in problem_5.J) <= problem_5.PG[3, 1] * sum(problem_5.X[3, j] for j in problem_5.J))

problem_5.r7 = Constraint(expr=sum(problem_5.X[i, j] * problem_5.CostoC[j] for i in problem_5.I for j in problem_5.J) <= problem_5.Presu)

problem_5.r8 = Constraint(expr=sum(problem_5.X[i, 1] for i in problem_5.I) >= problem_5.BA)

problem_5.r9 = ConstraintList()
for i in [1, 2]:
    problem_5.r9.add(sum(problem_5.X[i, j] for j in problem_5.J) >= problem_5.DEM[i])


# Usando gurobi como solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_5)

# Print the optimal solution
print("Objective Value:", problem_5.z())
print("X:")
for i in problem_5.I:
    for j in problem_5.J:
        print(i, j, problem_5.X[i, j].value)

Objective Value: 165.6696428571429
X:
1 1 25.71428571428571
1 2 34.285714285714285
1 3 0.0
2 1 35.0
2 2 35.0
2 3 0.0
3 1 0.0
3 2 82.83482142857144
3 3 82.83482142857144


In [34]:
problem_6 = ConcreteModel()

# define sets I 1..4
problem_6.I = RangeSet(1,4)

# define sets J 1..4
problem_6.J = RangeSet(1,4)

# fixed cost dictionary 3,4
fixed_cost = { 3: 1000, 4: 900}

# param fixed_cost
problem_6.CF = Param(RangeSet(3,4), initialize=fixed_cost)

# transportation cost dictionary
transportation_cost = { (1,1): 6, (1,2): 4, (1,3): 2, (1,4): 6,
                        (2,1): 2, (2,2): 3, (2,3): 7, (2,4): 5,
                        (3,1): 3, (3,2): 2, (3,3): 4, (3,4): 8,
                        (4,1): 6, (4,2): 3, (4,3): 4, (4,4): 2}

# param transportation_cost
problem_6.CT = Param(problem_6.I, problem_6.J, initialize=transportation_cost)

# warehouse demand dictionary
warehouse_demand = { 1: 700*1.25, 2: 800*1.25, 3: 500*1.25, 4: 400*1.25}

# param warehouse_demand
problem_6.DEM = Param(problem_6.J, initialize=warehouse_demand)

# Warehouse capacity dictionary
warehouse_capacity = { 1: 900, 2: 1500, 3: 900, 4: 900}

# param warehouse_capacity
problem_6.CAP = Param(problem_6.I, initialize=warehouse_capacity)

# define variables x[i,j] >= 0 integer
problem_6.x = Var(problem_6.I, problem_6.J, domain=NonNegativeIntegers)

# define variables y[i] binary 3,4
problem_6.y = Var(RangeSet(3,4), domain=Binary)

# define objective function minimize sum{i in I, j in J} CT[i,j] * x[i,j] + sum{i in I} CF[i] * y[i]
problem_6.z = Objective(expr=sum(problem_6.CT[i,j] * problem_6.x[i,j] for i in problem_6.I for j in problem_6.J) + sum(problem_6.CF[i] * problem_6.y[i] for i in RangeSet(3,4)), sense=minimize)

# define constraint sum{i in I} x[i,j] >= DEM[j] for j in J
problem_6.r1 = ConstraintList()
for j in problem_6.J:
    problem_6.r1.add(sum(problem_6.x[i,j] for i in problem_6.I) >= problem_6.DEM[j])

# define constraint sum{j in J} x[i,j] <= CAP[i] for i in 1,2
problem_6.r2 = ConstraintList()
for i in {1,2}:
    problem_6.r2.add(sum(problem_6.x[i,j] for j in problem_6.J) <= problem_6.CAP[i])

# define constraint sum{j in J} x[i,j] <= CAP[i] * y[i] for i in 3,4
problem_6.r3 = ConstraintList()
for i in {3,4}:
    problem_6.r3.add(sum(problem_6.x[i,j] for j in problem_6.J) <= problem_6.CAP[i] * problem_6.y[i])

# define constraint sum{i in {3.4}} y[i] = 1
problem_6.r4 = Constraint(expr=sum(problem_6.y[i] for i in RangeSet(3,4)) == 1)

# solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_6)

print('Solución optima:')
print('Valor óptimo:', value(problem_6.z))
for x in problem_6.x:
    print(x, value(problem_6.x[x]))
print('Valor óptimo:', value(problem_6.z))

Solución optima:
Valor óptimo: 7900.0
(1, 1) -0.0
(1, 2) -0.0
(1, 3) 625.0
(1, 4) -0.0
(2, 1) 875.0
(2, 2) 625.0
(2, 3) -0.0
(2, 4) -0.0
(3, 1) -0.0
(3, 2) -0.0
(3, 3) -0.0
(3, 4) -0.0
(4, 1) -0.0
(4, 2) 375.0
(4, 3) -0.0
(4, 4) 500.0
Valor óptimo: 7900.0


In [37]:
problem_7 = ConcreteModel()

# define sets I 1..5
problem_7.I = RangeSet(1,5)

# define sets J 1..3
problem_7.J = RangeSet(1,3)

# define sets T 1..3
problem_7.T = RangeSet(1,3)

# variable cost dictionary C{I,J}
variable_cost = { (1,1): 250, (1,2): 125, (1,3): 200,
                (2,1): 125, (2,2): 300, (2,3): 350,
                (3,1): 500, (3,2): 450, (3,3): 600,
                (4,1): 100, (4,2): 200, (4,3): 125,
                (5,1): 250, (5,2): 400, (5,3): 300}

# param C{I,J}
problem_7.CF = Param(problem_7.I, problem_7.J, initialize=variable_cost)

# fixed cost dictionary CF{I,J}
fixed_cost = { (1,1): 600, (1,2): 700, (1,3): 800,
                (2,1): 200, (2,2): 150, (2,3): 300,
                (3,1): 800, (3,2): 1000, (3,3): 900,
                (4,1): 300, (4,2): 120, (4,3): 250,
                (5,1): 500, (5,2): 400, (5,3): 225}

# param CF{I,J}
problem_7.CT = Param(problem_7.I, problem_7.J, initialize=fixed_cost)

# extraction capacity dictionary Cap{J,T}
extraction_capacity = { (1,1): 600, (1,2): 200, (1,3): 800,
                        (2,1): 900, (2,2): 300, (2,3): 500,
                        (3,1): 400, (3,2): 800, (3,3): 300}

# param Cap{J,T}
problem_7.Cap = Param(problem_7.J, problem_7.T, initialize=extraction_capacity)

# Demand dictionary Dem{I,T}
demand = { (1,1): 50, (1,2): 75, (1,3): 80,
            (2,1): 25, (2,2): 8, (2,3): 17,
            (3,1): 9, (3,2): 10, (3,3): 5,
            (4,1): 70, (4,2): 85, (4,3): 52,
            (5,1): 60, (5,2): 26, (5,3): 35}

# param Dem{I,T}
problem_7.Dem = Param(problem_7.I, problem_7.T, initialize=demand)

# param M
problem_7.M = Param(initialize=1000000)

# define variables x[I,J,T] >= 0
problem_7.x = Var(problem_7.I, problem_7.J, problem_7.T, domain=NonNegativeReals)

# define variables y[I,J] binary
problem_7.y = Var(problem_7.I, problem_7.J, domain=Binary)

# define objective function sum{i in I, j in J, t in T} C[i,j] * x[i,j,t] + sum{i in I, j in J} CF[i,j] * y[i,j]
problem_7.z = Objective(expr=sum(problem_7.CF[i,j] * problem_7.x[i,j,t] for i in problem_7.I for j in problem_7.J for t in problem_7.T) + sum(problem_7.CT[i,j] * problem_7.y[i,j] for i in problem_7.I for j in problem_7.J), sense=minimize)

# define constraint sum{j in J} x[i,j,t] <= Cap[j,t] for j in J, t in T
problem_7.r1 = ConstraintList()
for j in problem_7.J:
    for t in problem_7.T:
        problem_7.r1.add(sum(problem_7.x[i,j,t] for i in problem_7.I) <= problem_7.Cap[j,t])

# define constraint sum{j in J} x[i,j,t] <= Dem[i,t] for i in I, t in T
problem_7.r2 = ConstraintList()
for i in problem_7.I:
    for t in problem_7.T:
        problem_7.r1.add(sum(problem_7.x[i,j,t] for j in problem_7.J) >= problem_7.Dem[i,t])

# define constraint sum{t in T} x[i,j,t] <= M * y[i,j] for i in I, j in J
problem_7.r3 = ConstraintList()
for i in problem_7.I:
    for j in problem_7.J:
        problem_7.r1.add(sum(problem_7.x[i,j,t] for t in problem_7.T) <= problem_7.M * problem_7.y[i,j])


solver = SolverFactory('gurobi')
results = solver.solve(problem_7)

print('Solución optima:')
for x in problem_7.x:
    print(x, value(problem_7.x[x]))
print('Valor óptimo:', value(problem_7.z))

Solución optima:
(1, 1, 1) 0.0
(1, 1, 2) 0.0
(1, 1, 3) 0.0
(1, 2, 1) 50.0
(1, 2, 2) 75.0
(1, 2, 3) 80.0
(1, 3, 1) 0.0
(1, 3, 2) 0.0
(1, 3, 3) 0.0
(2, 1, 1) 25.0
(2, 1, 2) 8.0
(2, 1, 3) 17.0
(2, 2, 1) 0.0
(2, 2, 2) 0.0
(2, 2, 3) 0.0
(2, 3, 1) 0.0
(2, 3, 2) 0.0
(2, 3, 3) 0.0
(3, 1, 1) 0.0
(3, 1, 2) 0.0
(3, 1, 3) 0.0
(3, 2, 1) 9.0
(3, 2, 2) 10.0
(3, 2, 3) 5.0
(3, 3, 1) 0.0
(3, 3, 2) 0.0
(3, 3, 3) 0.0
(4, 1, 1) 70.0
(4, 1, 2) 85.0
(4, 1, 3) 52.0
(4, 2, 1) 0.0
(4, 2, 2) 0.0
(4, 2, 3) 0.0
(4, 3, 1) 0.0
(4, 3, 2) 0.0
(4, 3, 3) 0.0
(5, 1, 1) 60.0
(5, 1, 2) 26.0
(5, 1, 3) 35.0
(5, 2, 1) 0.0
(5, 2, 2) 0.0
(5, 2, 3) 0.0
(5, 3, 1) 0.0
(5, 3, 2) 0.0
(5, 3, 3) 0.0
Valor óptimo: 96325.0


In [38]:
problem_8 = ConcreteModel()

# define sets I 0..7
problem_8.I = RangeSet(0,7)

# define sets A {i in I, j in I: i != j}
problem_8.A = Set(within=problem_8.I * problem_8.I, initialize=lambda model: [(i,j) for i in model.I for j in model.I if i != j])

# cost of moving from i to j dictionary c{i,j}
cost = { (0,1): 8, (0,2): 7, (0,3): 999, (0,4): 4, (0,5): 999, (0,6): 999, (0,7): 999,
        (1,0): 3, (1,2): 6, (1,3): 8, (1,4): 7, (1,5): 4, (1,6): 6, (1,7): 999,
        (2,0): 6, (2,1): 10, (2,3): 8, (2,4): 7, (2,5): 2, (2,6): 2, (2,7): 999,
        (3,0): 999, (3,1): 2, (3,2): 1, (3,4): 4, (3,5): 9, (3,6): 7, (3,7): 999,
        (4,0): 10, (4,1): 1, (4,2): 5, (4,3): 3, (4,5): 1, (4,6): 10, (4,7): 1,
        (5,0): 999, (5,1): 9, (5,2): 3, (5,3): 9, (5,4): 2, (5,6): 2, (5,7): 6,
        (6,0): 999, (6,1): 3, (6,2): 5, (6,3): 10, (6,4): 5, (6,5): 2, (6,7): 6,
        (7,0): 999, (7,1): 999, (7,2): 999, (7,3): 999, (7,4): 10, (7,5): 8, (7,6): 3}

# param c{{i,j} in A}
problem_8.c = Param(problem_8.A, initialize=cost)

# cost of opening door i dictionary door{I}
door = {0: 0, 1: 6, 2: 8, 3: 8, 4: 2, 5: 7, 6: 6, 7: 3}

# param door{I}
problem_8.door = Param(problem_8.I, initialize=door)

# define variables x{I,I} binary
problem_8.x = Var(problem_8.A, domain=Binary)

# define variables y{I} binary
problem_8.y = Var(problem_8.I, domain=Binary)

# define objective function sum{(i,j) in A} c[i,j] * x[i,j] + sum{i in I} door[i] * y[i]
problem_8.z = Objective(expr=sum(problem_8.c[i,j] * problem_8.x[i,j] for (i,j) in problem_8.A) + sum(problem_8.door[i] * problem_8.y[i] for i in problem_8.I), sense=minimize)

# define constraint sum{i in I diff {0}} x[0,i] = 1
problem_8.r1 = Constraint(expr=sum(problem_8.x[0,i] for i in problem_8.I if i != 0) == 1)

# define constraint sum{i in I diff {7}} x[i,7] = 1
problem_8.r2 = Constraint(expr=sum(problem_8.x[i,7] for i in problem_8.I if i != 7) == 1)

# define constraint sum{i in I: i!=j} x[i,j] = sum{i in I: i!=j} x[j,i] for j in I diff {0,7}
problem_8.r3 = ConstraintList()
for j in problem_8.I:
    if j != 0 and j != 7:
        problem_8.r3.add(sum(problem_8.x[i,j] for i in problem_8.I if i != j) == sum(problem_8.x[j,i] for i in problem_8.I if i != j))

# define constraint y[i] >= sum{j in I: j!=i} x[i,j] for i in I
problem_8.r4 = ConstraintList()
for i in problem_8.I:
    problem_8.r4.add(problem_8.y[i] >= sum(problem_8.x[i,j] for j in problem_8.I if j != i))

solver = SolverFactory('gurobi')
results = solver.solve(problem_8)

print('Solución optima:')
for x in problem_8.x:
    print(x, value(problem_8.x[x]))
print('Valor óptimo:', value(problem_8.z))

Solución optima:
(0, 1) -0.0
(0, 2) -0.0
(0, 3) -0.0
(0, 4) 1.0
(0, 5) -0.0
(0, 6) -0.0
(0, 7) -0.0
(1, 0) -0.0
(1, 2) -0.0
(1, 3) -0.0
(1, 4) -0.0
(1, 5) -0.0
(1, 6) -0.0
(1, 7) 0.0
(2, 0) -0.0
(2, 1) -0.0
(2, 3) -0.0
(2, 4) -0.0
(2, 5) -0.0
(2, 6) -0.0
(2, 7) 0.0
(3, 0) 0.0
(3, 1) -0.0
(3, 2) -0.0
(3, 4) -0.0
(3, 5) -0.0
(3, 6) -0.0
(3, 7) 0.0
(4, 0) -0.0
(4, 1) -0.0
(4, 2) -0.0
(4, 3) -0.0
(4, 5) -0.0
(4, 6) -0.0
(4, 7) 1.0
(5, 0) 0.0
(5, 1) -0.0
(5, 2) -0.0
(5, 3) -0.0
(5, 4) -0.0
(5, 6) -0.0
(5, 7) -0.0
(6, 0) 0.0
(6, 1) -0.0
(6, 2) -0.0
(6, 3) -0.0
(6, 4) -0.0
(6, 5) -0.0
(6, 7) -0.0
(7, 0) 0.0
(7, 1) 0.0
(7, 2) 0.0
(7, 3) 0.0
(7, 4) -0.0
(7, 5) -0.0
(7, 6) -0.0
Valor óptimo: 7.0


In [47]:
problem_9 = ConcreteModel()

# define set T 1..5
problem_9.T = RangeSet(1,5)

demand = {1: 100, 2: 200, 3: 300, 4: 400, 5: 200}

#Param D{T}
problem_9.D = Param(problem_9.T, initialize=demand)

#Param Ch
problem_9.Ch = Param(initialize=10)

#Param Cf
problem_9.Cf = Param(initialize=30)

#Param C1
problem_9.C1 = Param(initialize=100)

#Param C2
problem_9.C2 = Param(initialize=100)

#Param Num
problem_9.Num = Param(initialize=8)

# define variables x{T union {0}} >= 0 integer
problem_9.x = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)

# define variable I{T union {0}} >= 0 integer
problem_9.I = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)

# define variable y{T union {0}} >= 0 integer
problem_9.y = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)

# define variable z{T union {0}} >= 0 integer
problem_9.z = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)

# define variable m{T union {0}} >= 0 integer
problem_9.m = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)

# define objective function z sum{t in T} (C1 * (y[t] + z[t] + (z[t]/5)) + C2 * m[t] + Ch * I[t] + Cf * x[t])
problem_9.obj = Objective(expr=sum(problem_9.C1 * (problem_9.y[t] + problem_9.z[t] + (problem_9.z[t]/5)) + problem_9.C2 * problem_9.m[t] + problem_9.Ch * problem_9.I[t] + problem_9.Cf * problem_9.x[t] for t in problem_9.T), sense=minimize)

# define constraint  I[t-1] + y[t] * Num = D[t] + x[t-1] + I[t] - x[t] for t in T
problem_9.r1 = ConstraintList()
for t in problem_9.T:
    problem_9.r1.add(problem_9.I[t-1] + problem_9.y[t] * problem_9.Num == problem_9.D[t] + problem_9.x[t-1] + problem_9.I[t] - problem_9.x[t])
   
# define constraint y[t] = y[t-1] - m[t-1] + z[t-1] - z[t]/5 + z[t-1]/5 for t in T
problem_9.r2 = ConstraintList()
for t in problem_9.T:
    problem_9.r2.add(problem_9.y[t] == problem_9.y[t-1] - problem_9.m[t-1] + problem_9.z[t-1] - problem_9.z[t]/5 + problem_9.z[t-1]/5)

# define constraint x[0] = 0
problem_9.r3 = Constraint(expr=problem_9.x[0] == 0)

# define constraint I[0] = 10
problem_9.r4 = Constraint(expr=problem_9.I[0] == 10)

# define constraint y[0] = 20
problem_9.r5 = Constraint(expr=problem_9.y[0] == 20)

# define constraint z[0] = 0
problem_9.r6 = Constraint(expr=problem_9.z[0] == 0)

# define constraint m[0] = 0
problem_9.r7 = Constraint(expr=problem_9.m[0] == 0)

solver = SolverFactory('gurobi')
results = solver.solve(problem_9)

print('Solución optima:')
# print as matrix
for t in problem_9.T:
    print(t, sep=' ', end=' ')
print('')
print('Puertas no entregadas', end=' ')
for x in problem_9.x:
    print(value(problem_9.x[x]), sep=' ', end=' ')
print('')
print('Puertas en inventario', end=' ')
for i in problem_9.I:
    print(value(problem_9.I[i]), sep=' ', end=' ')
print('')
print('Expertos en producción', end=' ')
for y in problem_9.y:
    print(value(problem_9.y[y]), sep=' ', end=' ')
print('')
print('Novatos en capacitación', end=' ')
for z in problem_9.z:
    print(value(problem_9.z[z]), sep=' ', end=' ')
print('')
print('Operarios despedidos', end=' ')
for m in problem_9.m:
    print(value(problem_9.m[m]), sep=' ', end=' ')
print('')
print('Valor óptimo:', value(problem_9.obj))


Solución optima:
1 2 3 4 5 
Puertas no entregadas -0.0 -0.0 -0.0 54.0 -0.0 0.0 
Puertas en inventario 54.0 86.0 66.0 -0.0 2.0 10.0 
Expertos en producción 18.0 29.0 35.0 35.0 32.0 20.0 
Novatos en capacitación 10.0 5.0 0.0 0.0 0.0 0.0 
Operarios despedidos 0.0 -0.0 0.0 3.0 -0.0 0.0 
Valor óptimo: 20700.0


In [40]:
problem_9p = ConcreteModel()

# define set I from 1 to 5
problem_9p.I = RangeSet(1,5)

# define set J from 1 to 4
problem_9p.J = RangeSet(1,4)

# utility dictionary
utility = {1: 50, 2: 65, 3: 40, 4: 70, 5: 55}

# define parameter u{I}
problem_9p.u = Param(problem_9p.I, initialize=utility)

# demand dictionary
demand = {1: 15, 2: 12, 3: 32, 4: 18, 5: 10}

# define parameter d{I}
problem_9p.d = Param(problem_9p.I, initialize=demand)

# dish ingredients requirement dictionary
dish_ingredients = {(1,1): 1, (1,2): 1, (1,3): 0, (1,4): 1,
                    (2,1): 4, (2,2): 2, (2,3): 0, (2,4): 1,
                    (3,1): 2, (3,2): 1, (3,3): 1, (3,4): 2,
                    (4,1): 0, (4,2): 0, (4,3): 2, (4,4): 0,
                    (5,1): 1, (5,2): 1, (5,3): 0, (5,4): 1}

# define parameter a{I,J}
problem_9p.a = Param(problem_9p.I, problem_9p.J, initialize=dish_ingredients)

# available ingredients dictionary
fresh_ingredients = {1: 80, 2: 75, 3: 10, 4: 30}

# define parameter m{J}
problem_9p.m = Param(problem_9p.J, initialize=fresh_ingredients)

# define variable x positive integer for each element in I
problem_9p.x = Var(problem_9p.I, domain=NonNegativeIntegers)

# define objective function sum i in I u[i] * x[i]
problem_9p.z = Objective(expr=sum(problem_9p.u[i] * problem_9p.x[i] for i in problem_9p.I), sense=maximize)

# define constraint sum i in I a[i,j] * x[i] <= m[j] for each j in J
problem_9p.r1 = ConstraintList()
for j in problem_9p.J:
    problem_9p.r1.add(sum(problem_9p.a[i,j] * problem_9p.x[i] for i in problem_9p.I) <= problem_9p.m[j])

# define constraint x[i] >= d[i] for each i in I
problem_9p.r2 = ConstraintList()
for i in problem_9p.I:
    problem_9p.r2.add(problem_9p.x[i] <= problem_9p.d[i])

# define constraint each of the dishes must be prepared at least 3 times
problem_9p.r3 = ConstraintList()
for i in problem_9p.I:
    problem_9p.r3.add(problem_9p.x[i] >= 3)
    
solver = SolverFactory('gurobi')
results = solver.solve(problem_9p)

print('Solución optima:')
for x in problem_9p.x:
    print(x, value(problem_9p.x[x]))
print('Valor óptimo:', value(problem_9p.z))


Solución optima:
1 3.0
2 12.0
3 3.0
4 3.0
5 9.0
Valor óptimo: 1755.0


In [55]:
problem_10 = ConcreteModel()

# define set I 1,2
problem_10.I = RangeSet(1,2)

# define set J 1..4
problem_10.J = RangeSet(1,4)

# define set K 1..3
problem_10.K = RangeSet(1,3)

# Param M
problem_10.M = Param(initialize=1000000)

max_cap = {1: 5000, 2: 3000, 3: 4000}

# Param Cmax{K}
problem_10.Cmax = Param(problem_10.K, initialize=max_cap)


# percentage of metal extracted from each mine dictionary
mper_perton_permine = {
    (1, 1): 15, (2, 1): 20, (3, 1): 40, (4, 1): 25,
    (1, 2): 10, (2, 2): 30, (3, 2): 10, (4, 2): 40,
    (1, 3): 5, (2, 3): 10, (3, 3): 70, (4, 3): 10
}

# Param Por{J,K}
problem_10.Por = Param(problem_10.J, problem_10.K, initialize=mper_perton_permine)

cost_ton = {1: 70, 2: 50, 3: 65}

# Param CT{K}
problem_10.CT = Param(problem_10.K, initialize=cost_ton)

sale_price = {1: 250, 2: 170}

# Param PV{I}
problem_10.PV = Param(problem_10.I, initialize=sale_price)

# var X{K,I} >= 0
problem_10.X = Var(problem_10.K, problem_10.I, domain=NonNegativeReals)

# var Y{K diff {3}} binary
problem_10.Y = Var(problem_10.K - {3}, domain=Binary)

# var Z binary
problem_10.Z = Var(domain=Binary)

# max z sum {i in I, k in K} PV[i] * X[k,i] - sum {i in I,k in K} CT[k] * X[k,i]
problem_10.obj = Objective(expr=sum(problem_10.PV[i] * problem_10.X[k,i] for i in problem_10.I for k in problem_10.K) - sum(problem_10.CT[k] * problem_10.X[k,i] for i in problem_10.I for k in problem_10.K), sense=maximize)

# define constraint sum {i in I} X[k,i] <= Cmax[k] * Y[k] for each k in {1,2}
problem_10.r1 = ConstraintList()
for k in problem_10.K - {3}:
    problem_10.r1.add(sum(problem_10.X[k,i] for i in problem_10.I) <= problem_10.Cmax[k] * problem_10.Y[k])

# define constraint sum {i in I} X[k,i] <= Cmax[k] for k in {3}
problem_10.r2 = ConstraintList()
for k in {3}:
    problem_10.r2.add(sum(problem_10.X[k,i] for i in problem_10.I) <= problem_10.Cmax[k])

# define constraint sum {k in K} x[k,1] * Por[1,k] <= sum{k in K} x[k,1] * 0.4
problem_10.r3 = Constraint(expr=sum(problem_10.X[k,1] * problem_10.Por[1,k] for k in problem_10.K) <= sum(problem_10.X[k,1] * 0.4 for k in problem_10.K))

# define constraint sum {k in K} x[k,1] * Por[2,k] <= sum{k in K} x[k,1] * 0.75
problem_10.r4 = Constraint(expr=sum(problem_10.X[k,1] * problem_10.Por[2,k] for k in problem_10.K) <= sum(problem_10.X[k,1] * 0.75 for k in problem_10.K))

# define constraint sum {k in K} x[k,1] * Por[4,k] >= sum{k in K} x[k,1] * 0.3
problem_10.r5 = Constraint(expr=sum(problem_10.X[k,1] * problem_10.Por[4,k] for k in problem_10.K) >= sum(problem_10.X[k,1] * 0.3 for k in problem_10.K))

# define constraint sum {k in K} x[k,2] * Por[2,k] >= sum{k in K} x[k,2] * 0.25
problem_10.r6 = Constraint(expr=sum(problem_10.X[k,2] * problem_10.Por[2,k] for k in problem_10.K) >= sum(problem_10.X[k,2] * 0.25 for k in problem_10.K))

# define constraint sum {k in K} x[k,2] * Por[2,k] <= sum{k in K} x[k,2] * 0.8
problem_10.r7 = Constraint(expr=sum(problem_10.X[k,2] * problem_10.Por[2,k] for k in problem_10.K) <= sum(problem_10.X[k,2] * 0.8 for k in problem_10.K))

# define constraint sum {k in K} x[k,2] * Por[3,k] >= sum{k in K} x[k,2] * 0.5
problem_10.r8 = Constraint(expr=sum(problem_10.X[k,2] * problem_10.Por[3,k] for k in problem_10.K) >= sum(problem_10.X[k,2] * 0.5 for k in problem_10.K))

# define constraint sum {k in K} x[k,2] * Por[4,k] <= sum{k in K} x[k,2] * 0.66
problem_10.r9 = Constraint(expr=sum(problem_10.X[k,2] * problem_10.Por[4,k] for k in problem_10.K) <= sum(problem_10.X[k,2] * 0.66 for k in problem_10.K))

# define constraint Y[1] = 1 - Y[2]
problem_10.r10 = Constraint(expr=problem_10.Y[1] == 1 - problem_10.Y[2])

# define constraint X[3,1] <=  Z * M
problem_10.r11 = Constraint(expr=problem_10.X[3,1] <= problem_10.Z * problem_10.M)

# define constraint X[3,2] <=  (1-Z) * M
problem_10.r12 = Constraint(expr=problem_10.X[3,2] <= (1 - problem_10.Z) * problem_10.M)

solver = SolverFactory('gurobi')
results = solver.solve(problem_10)

print('Solución optima:')
for x in problem_10.X:
    print(x, value(problem_10.X[x]))
print('Valor óptimo:', value(problem_10.obj))

Solución optima:
(1, 1) 0.0
(1, 2) 0.0
(2, 1) 0.0
(2, 2) 0.0
(3, 1) 0.0
(3, 2) 0.0
Valor óptimo: 0.0


In [59]:
problem_11 = ConcreteModel()

# define set I 1,2
problem_11.I = RangeSet(1,2)

# define set J 1,2,3
problem_11.J = RangeSet(1,3)

# define set K 1,2
problem_11.K =RangeSet(1,2)

demand = {1: 3000, 2: 2000}

# param Dem{I}
problem_11.Dem = Param(problem_11.I, initialize=demand)


# capacity dictinary C{I,J,K}
capacity = {(1,1,1): 0.5, (1,1,2): 0.6, (1,2,1): 3.75, (1,2,2): 4, (1,3,1): 0.6, (1,3,2): 0.65,
            (2,1,1): 0.5, (2,1,2): 0.6, (2,2,1): 3.3, (2,2,2): 3.9, (2,3,1): 0.75, (2,3,2): 0.78}

# param C{I,J,K}
problem_11.C = Param(problem_11.I, problem_11.J, problem_11.K, initialize=capacity)

# param CapH
problem_11.CapH = Param(initialize=200)

# param CapHE
problem_11.CapHE = Param(initialize=50)

# param CHE
problem_11.CHE = Param(initialize=9)

timeontask = {(1,1): 1, (1,2): 3, (1,3): 1, (2,1): 1, (2,2): 2.5, (2,3): 1.5}

# param T{I,J}
problem_11.T = Param(problem_11.I, problem_11.J, initialize=timeontask)

# var X{I,J,K} >= 0 integer
problem_11.X = Var(problem_11.I, problem_11.J, problem_11.K, within=NonNegativeIntegers)

# var H >= 0
problem_11.H = Var(within=NonNegativeReals)

# minimize sum {i in I, j in J, k in K} C[i,j,k] * X[i,j,k] + CHE * H
problem_11.obj = Objective(expr=sum(problem_11.C[i,j,k] * problem_11.X[i,j,k] for i in problem_11.I for j in problem_11.J for k in problem_11.K) + problem_11.CHE * problem_11.H, sense=minimize)

# define constraint sum {k in K} X[i,j,k] >= Dem[1] for all i in I, j in J
problem_11.r1 = ConstraintList()
for i in problem_11.I:
    for j in problem_11.J:
        problem_11.r1.add(sum(problem_11.X[i,j,k] for k in problem_11.K) >= problem_11.Dem[i])

# define constraint H <= CapHE
problem_11.r2 = Constraint(expr=problem_11.H <= problem_11.CapHE)

# define constraint sum {i in I, j in J} (T[i,j]/60) * X[i,j,1] <= H + CapH
problem_11.r3 = Constraint(expr=sum((problem_11.T[i,j]/60) * problem_11.X[i,j,1] for i in problem_11.I for j in problem_11.J) <= problem_11.H + problem_11.CapH)


solver = SolverFactory('gurobi')
results = solver.solve(problem_11)

print('Solución optima:')
for x in problem_11.X:
    print(x, value(problem_11.X[x]))
print('Valor óptimo:', value(problem_11.obj))


Solución optima:
(1, 1, 1) 3000.0
(1, 1, 2) -0.0
(1, 2, 1) 666.0
(1, 2, 2) 2334.0
(1, 3, 1) 1.0
(1, 3, 2) 2999.0
(2, 1, 1) 2000.0
(2, 1, 2) -0.0
(2, 2, 1) 2000.0
(2, 2, 2) -0.0
(2, 3, 1) -0.0
(2, 3, 2) 2000.0
Valor óptimo: 24443.45


In [60]:
problem_12 = ConcreteModel()

# define set I 1..5
problem_12.I = RangeSet(1,5)

# define set J 1..4
problem_12.J = RangeSet(1,4)

# define set K 1..2
problem_12.K = RangeSet(1,2)

cost = {1: 50, 2: 30, 3: 35, 4: 60, 5: 40}

# Param C{I}
problem_12.C = Param(problem_12.I, initialize=cost)

demand = {(1,1): 3, (1,2): 3, (2,1): 2, (2,2): 2, (3,1): 2, (3,2): 2, (4,1): 4, (4,2): 3}

# Param DEM{J,K}
problem_12.DEM = Param(problem_12.J, problem_12.K, initialize=demand)

# var x{I,J,K} >= 0 integer
problem_12.x = Var(problem_12.I, problem_12.J, problem_12.K, within=NonNegativeIntegers)

# minimize sum {i in I, j in J, k in K} C[i] * x[i,j,k]
problem_12.z = Objective(expr=sum(problem_12.C[i] * problem_12.x[i,j,k] for i in problem_12.I for j in problem_12.J for k in problem_12.K), sense=minimize)

# define constraint sum {i in I} x[i,j,k] >= DEM[j] for all j in J and k in K
problem_12.r1 = ConstraintList()
for j in problem_12.J:
    for k in problem_12.K:
        problem_12.r1.add(sum(problem_12.x[i,j,k] for i in problem_12.I) >= problem_12.DEM[j,k])

# define constraint x[i,4,k] >= x[i,1,k] for all i in I and k in K
problem_12.r2 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r2.add(problem_12.x[i,4,k] >= problem_12.x[i,1,k])

# define constraint x[i,2,k] + x[i,3,k] <= 1 for all i in I and k in K
problem_12.r3 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r3.add(problem_12.x[i,2,k] + problem_12.x[i,3,k] <= 1)

# define constraint sum {j in {1..3}} x[i,j,k] <= 2 for all i in I and k in K
problem_12.r4 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r4.add(sum(problem_12.x[i,j,k] for j in range(1,4)) <= 2)

# define constraint sum {j in {2..4}} x[i,j,k] <= 2 for all i in I and k in K
problem_12.r5 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r5.add(sum(problem_12.x[i,j,k] for j in range(2,5)) <= 2)

solver = SolverFactory('gurobi')
results = solver.solve(problem_12)

print('Solución optima:')
for x in problem_12.x:
    print(x, value(problem_12.x[x]))
print('Valor óptimo:', value(problem_12.z))

Solución optima:
(1, 1, 1) -0.0
(1, 1, 2) -0.0
(1, 2, 1) 1.0
(1, 2, 2) -0.0
(1, 3, 1) 0.0
(1, 3, 2) 1.0
(1, 4, 1) 1.0
(1, 4, 2) -0.0
(2, 1, 1) 1.0
(2, 1, 2) 1.0
(2, 2, 1) 1.0
(2, 2, 2) -0.0
(2, 3, 1) -0.0
(2, 3, 2) 1.0
(2, 4, 1) 1.0
(2, 4, 2) 1.0
(3, 1, 1) 1.0
(3, 1, 2) 1.0
(3, 2, 1) 0.0
(3, 2, 2) 1.0
(3, 3, 1) 1.0
(3, 3, 2) 0.0
(3, 4, 1) 1.0
(3, 4, 2) 1.0
(4, 1, 1) -0.0
(4, 1, 2) -0.0
(4, 2, 1) -0.0
(4, 2, 2) -0.0
(4, 3, 1) -0.0
(4, 3, 2) -0.0
(4, 4, 1) -0.0
(4, 4, 2) -0.0
(5, 1, 1) 1.0
(5, 1, 2) 1.0
(5, 2, 1) 0.0
(5, 2, 2) 1.0
(5, 3, 1) 1.0
(5, 3, 2) 0.0
(5, 4, 1) 1.0
(5, 4, 2) 1.0
Valor óptimo: 780.0


In [66]:
problem_12 = ConcreteModel()

# define set I 1..5
problem_12.I = RangeSet(1,5)

# define set J 1..4
problem_12.J = RangeSet(1,4)

# define set K 1..2
problem_12.K = RangeSet(1,2)

cost = {1: 50, 2: 30, 3: 35, 4: 60, 5: 40}

# Param C{I}
problem_12.C = Param(problem_12.I, initialize=cost)

demand = {(1,1): 3, (1,2): 3, (2,1): 2, (2,2): 2, (3,1): 2, (3,2): 2, (4,1): 4, (4,2): 3}

# Param DEM{J,K}
problem_12.DEM = Param(problem_12.J, problem_12.K, initialize=demand)

# Var Shifts{I}
problem_12.Shifts = Var(problem_12.I, within=NonNegativeIntegers)

problem_12.Target = Var(within=NonNegativeIntegers)

# var x{I,J,K} >= 0 integer
problem_12.x = Var(problem_12.I, problem_12.J, problem_12.K, within=NonNegativeIntegers)

# minimize sum {i in I, j in J, k in K} C[i] * x[i,j,k]
problem_12.z = Objective(expr=sum(problem_12.C[i] * problem_12.x[i,j,k] for i in problem_12.I for j in problem_12.J for k in problem_12.K), sense=minimize)

# define constraint sum {i in I} x[i,j,k] >= DEM[j] for all j in J and k in K
problem_12.r1 = ConstraintList()
for j in problem_12.J:
    for k in problem_12.K:
        problem_12.r1.add(sum(problem_12.x[i,j,k] for i in problem_12.I) >= problem_12.DEM[j,k])

# define constraint x[i,4,k] >= x[i,1,k] for all i in I and k in K
problem_12.r2 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r2.add(problem_12.x[i,4,k] >= problem_12.x[i,1,k])

# define constraint x[i,2,k] + x[i,3,k] <= 1 for all i in I and k in K
problem_12.r3 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r3.add(problem_12.x[i,2,k] + problem_12.x[i,3,k] <= 1)

# define constraint sum {j in {1..3}} x[i,j,k] <= 2 for all i in I and k in K
problem_12.r4 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r4.add(sum(problem_12.x[i,j,k] for j in range(1,4)) <= 2)

# define constraint sum {j in {2..4}} x[i,j,k] <= 2 for all i in I and k in K
problem_12.r5 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r5.add(sum(problem_12.x[i,j,k] for j in range(2,5)) <= 2)

# Constraint to enforce equality in the number of shifts for each nurse
problem_12.r6 = Constraint(expr=sum(problem_12.Shifts[i] for i in problem_12.I) == problem_12.Target)

# Constraint to ensure each nurse gets the same number of shifts
problem_12.r7 = ConstraintList()
for i in problem_12.I:
    problem_12.r7.add(problem_12.Shifts[i] == problem_12.Target)

solver = SolverFactory('gurobi')
results = solver.solve(problem_12)

print('Solución optima:')
for x in problem_12.x:
    print(x, value(problem_12.x[x]))
print('Valor óptimo:', value(problem_12.z))

Solución optima:
(1, 1, 1) -0.0
(1, 1, 2) -0.0
(1, 2, 1) 1.0
(1, 2, 2) -0.0
(1, 3, 1) 0.0
(1, 3, 2) 1.0
(1, 4, 1) 1.0
(1, 4, 2) -0.0
(2, 1, 1) 1.0
(2, 1, 2) 1.0
(2, 2, 1) 1.0
(2, 2, 2) -0.0
(2, 3, 1) -0.0
(2, 3, 2) 1.0
(2, 4, 1) 1.0
(2, 4, 2) 1.0
(3, 1, 1) 1.0
(3, 1, 2) 1.0
(3, 2, 1) 0.0
(3, 2, 2) 1.0
(3, 3, 1) 1.0
(3, 3, 2) 0.0
(3, 4, 1) 1.0
(3, 4, 2) 1.0
(4, 1, 1) -0.0
(4, 1, 2) -0.0
(4, 2, 1) -0.0
(4, 2, 2) -0.0
(4, 3, 1) -0.0
(4, 3, 2) -0.0
(4, 4, 1) -0.0
(4, 4, 2) -0.0
(5, 1, 1) 1.0
(5, 1, 2) 1.0
(5, 2, 1) 0.0
(5, 2, 2) 1.0
(5, 3, 1) 1.0
(5, 3, 2) 0.0
(5, 4, 1) 1.0
(5, 4, 2) 1.0
Valor óptimo: 780.0


In [69]:
problem_13 = ConcreteModel()

# define set I 1..4
problem_13.I = RangeSet(1,4)

# define set J 1..3
problem_13.J = RangeSet(1,3)

cost = {(1,1): 18, (1,2): 48, (1,3): 17, 
        (2,1): 47, (2,2): 10, (2,3): 48, 
        (3,1): 38, (3,2): 10, (3,3): 12, 
        (4,1): 20, (4,2): 50, (4,3): 35}

# Param c{I,J}
problem_13.c = Param(problem_13.I, problem_13.J, initialize=cost)

fab_cap = {1: 75, 2: 75, 3: 75, 4: 75}

# Param of{I}
problem_13.of = Param(problem_13.I, initialize=fab_cap)

demand = {1: 100, 2: 120, 3: 80}

# Param dem{J}
problem_13.dem = Param(problem_13.J, initialize=demand)

# Var x{I,J} >= 0 
problem_13.x = Var(problem_13.I, problem_13.J, within=NonNegativeReals)

# Var H{I,I} >= 0
problem_13.H = Var(problem_13.I, problem_13.I, within=NonNegativeReals)

# minimize sum {i in I, j in J} c[i,j] * x[i,j]
problem_13.z = Objective(expr=sum(problem_13.c[i,j] * problem_13.x[i,j] for i in problem_13.I for j in problem_13.J), sense=minimize)

# define constraint sum {i in I} x[i,j] >= dem[j] for all j in J
problem_13.r1 = ConstraintList()
for j in problem_13.J:
    problem_13.r1.add(sum(problem_13.x[i,j] for i in problem_13.I) >= problem_13.dem[j])

# define constraint sum {j in J} x[i,j] <= sum {k in I} H[k,i] for all i in I
problem_13.r2 = ConstraintList()
for i in problem_13.I:
    problem_13.r2.add(sum(problem_13.x[i,j] for j in problem_13.J) <= sum(problem_13.H[k,i] for k in problem_13.I))

# define constraint sum {k in I} H[i,k] <= of[i] for all i in I
problem_13.r3 = ConstraintList()
for i in problem_13.I:
    problem_13.r3.add(sum(problem_13.H[i,k] for k in problem_13.I) <= problem_13.of[i])

# define constraint sum {k in I} H[k,i] >= 10 for all i in I
problem_13.r4 = ConstraintList()
for i in problem_13.I:
    problem_13.r4.add(sum(problem_13.H[k,i] for k in problem_13.I) >= 10)

# solve
solver = SolverFactory('gurobi')
solver.solve(problem_13)

print('Solución optima:')
print('x[i,j] = ')
for i in problem_13.I:
    for j in problem_13.J:
        print('x[', i, ',', j, '] = ', value(problem_13.x[i,j]))
print('z = ', value(problem_13.z))




Solución optima:
x[i,j] = 
x[ 1 , 1 ] =  90.0
x[ 1 , 2 ] =  0.0
x[ 1 , 3 ] =  0.0
x[ 2 , 1 ] =  0.0
x[ 2 , 2 ] =  120.0
x[ 2 , 3 ] =  0.0
x[ 3 , 1 ] =  0.0
x[ 3 , 2 ] =  0.0
x[ 3 , 3 ] =  80.0
x[ 4 , 1 ] =  10.0
x[ 4 , 2 ] =  0.0
x[ 4 , 3 ] =  0.0
z =  3980.0


In [72]:
problem_14 = ConcreteModel()

# set I 1..2
problem_14.I = RangeSet(1,2)

# set J 1..2
problem_14.J = RangeSet(1,2)

# set K 1..2
problem_14.K = RangeSet(1,2)

# set M 1..2
problem_14.M = RangeSet(1,2)

sale_price_sw = {1: 200, 2: 150}

# Param Pe{I}
problem_14.Pe = Param(problem_14.I, initialize=sale_price_sw)

# Param Ph
problem_14.Ph = Param(initialize=180)

cost = {1: 25, 2: 15}

# Param C{M}
problem_14.C = Param(problem_14.M, initialize=cost)

req_sw = {1: 1, 2: 2}

# Param Me{M}
problem_14.Me = Param(problem_14.M, initialize=req_sw)

req_axe = {1: 2, 2: 1}

# Param Mh{M}
problem_14.Mh = Param(problem_14.M, initialize=req_axe)

# Param CapE
problem_14.CapE = Param(initialize=200)

# Param CapH
problem_14.CapH = Param(initialize=220)

# Param CapT
problem_14.CapT = Param(initialize=25)

timeontasksw = {(1,1): 6, (1,2): 2, (2,1): 0, (2,2): 3}

# Param Te{K,J}
problem_14.Te = Param(problem_14.K, problem_14.J, initialize=timeontasksw)

timeontaskaxe = {1: 3, 2: 2}

# Param Th{K}
problem_14.Th = Param(problem_14.K, initialize=timeontaskaxe)

# Param Horas
problem_14.Horas = Param(initialize=480)

reserve = {1: 600, 2: 500}

# Param Res{M}
problem_14.Res = Param(problem_14.M, initialize=reserve)

# Var E{I,J} >= 0
problem_14.E = Var(problem_14.I, problem_14.J, within=NonNegativeReals)

# Var H >= 0
problem_14.H = Var(within=NonNegativeReals)

# maximize sum {i in I, j in J} (Pe[i] * E[i,j]) + Ph * H - sum {m in M} (C[m] * (Me[m] * sum {i in I, j in J} E[i,j] + Mh[m] * H))
problem_14.z = Objective(expr=sum(problem_14.Pe[i] * problem_14.E[i,j] for i in problem_14.I for j in problem_14.J) + problem_14.Ph * problem_14.H - sum(problem_14.C[m] * (problem_14.Me[m] * sum(problem_14.E[i,j] for i in problem_14.I for j in problem_14.J) + problem_14.Mh[m] * problem_14.H) for m in problem_14.M), sense=maximize)

# define constraint (1/CapE) * sum {i in I, j in J} E[i,j] * (1/CapH) * H <= 1
problem_14.r1 = Constraint(expr=(1/problem_14.CapE) * sum(problem_14.E[i,j] for i in problem_14.I for j in problem_14.J) + (1/problem_14.CapH) * problem_14.H <= 1)

# define constraint sum {j in J} E[1,j] <= CapT
problem_14.r2 = Constraint(expr=sum(problem_14.E[1,j] for j in problem_14.J) <= problem_14.CapT)

# define constraint sum {j in J, i in I} Te[k,j] * E[i,j] + Th[k] * H <= Horas for all k in K
problem_14.r3 = ConstraintList()
for k in problem_14.K:
    problem_14.r3.add(sum(problem_14.Te[k,j] * problem_14.E[i,j] for j in problem_14.J for i in problem_14.I) + problem_14.Th[k] * problem_14.H <= problem_14.Horas)

# define constraint Me[m] * sum {i in I, j in J} E[i,j] + Mh[m] * H <= Res[m] for all m in M
problem_14.r4 = ConstraintList()
for m in problem_14.M:
    problem_14.r4.add(problem_14.Me[m] * sum(problem_14.E[i,j] for i in problem_14.I for j in problem_14.J) + problem_14.Mh[m] * problem_14.H <= problem_14.Res[m])


# solve
solver = SolverFactory('gurobi')
solver.solve(problem_14)

print('Solución optima:')
print ('H = ', value(problem_14.H))
print('E[i,j] = ')
for i in problem_14.I:
    for j in problem_14.J:
        print('E[', i, ',', j, '] = ', value(problem_14.E[i,j]))
print('z = ', value(problem_14.z))


Solución optima:
H =  96.0
E[i,j] = 
E[ 1 , 1 ] =  0.0
E[ 1 , 2 ] =  25.0
E[ 2 , 1 ] =  0.0
E[ 2 , 2 ] =  71.0
z =  21410.0


In [77]:
problem_15 = ConcreteModel()

# set I {A,B,C,D,E,F,G,H,I}
problem_15.I = Set(initialize=['A','B','C','D','E','F','G','H','I'])

# set J {1,2}
problem_15.J = Set(initialize=[1,2])

# Set Pred {{A,C},{A,D},{B,E},{C,E},{E,F},{D,G},{F,H},{G,I}}
problem_15.Pred = Set(initialize=[('A','C'),('A','D'),('B','E'),('C','E'),('E','F'),('D','G'),('F','H'),('G','I')])

# Param dueDate
problem_15.dueDate = Param(initialize=15)

timeontask = {('A',1): 1, ('A',2): 3, 
              ('B',1): 2, ('B',2): 4, 
              ('C',1): 0.5, ('C',2): 2, 
              ('D',1): 2, ('D',2): 5, 
              ('E',1): 1, ('E',2): 6, 
              ('F',1): 1, ('F',2): 2, 
              ('G',1): 3, ('G',2): 4, 
              ('H',1): 2, ('H',2): 3, 
              ('I',1): 4, ('I',2): 5}

# Param P{I,J}
problem_15.P = Param(problem_15.I, problem_15.J, initialize=timeontask)

cost = {'A': 4, 
        'B': 1, 
        'C': 1, 
        'D': 1, 
        'E': 3, 
        'F': 7, 
        'G': 9, 
        'H': 5, 
        'I': 8}

# Param costo{I}
problem_15.costo = Param(problem_15.I, initialize=cost)

# var x{I} >= 0 integer
problem_15.x = Var(problem_15.I, within=NonNegativeIntegers)

# var y{I} binary
problem_15.y = Var(problem_15.I, within=Binary)

# minimize sum {i in I} costo[i] * y[i]
problem_15.z = Objective(expr=sum(problem_15.costo[i] * problem_15.y[i] for i in problem_15.I), sense=minimize)

# define constraint x[i] + P[i,1] * (1-y[i]) + P[i,2] * y[i] <= dueDate for all i in I
problem_15.r1 = ConstraintList()
for i in problem_15.I:
    problem_15.r1.add(problem_15.x[i] + problem_15.P[i,1] * (1-problem_15.y[i]) + problem_15.P[i,2] * problem_15.y[i] <= problem_15.dueDate)

# define constraint x[k] >= x[i] + P[i,1] * (1-y[i]) + P[i,2] * y[i] for all (i,k) in Pred
problem_15.r2 = ConstraintList()
for i,k in problem_15.Pred:
    problem_15.r2.add(problem_15.x[k] >= problem_15.x[i] + problem_15.P[i,1] * (1-problem_15.y[i]) + problem_15.P[i,2] * problem_15.y[i])

# solve
solver = SolverFactory('gurobi')
solver.solve(problem_15)

print('Solución optima:')
for i in problem_15.I:
    print('x[', i, '] = ', value(problem_15.x[i]))
    print('y[', i, '] = ', value(problem_15.y[i]))
print('z = ', value(problem_15.z))


Solución optima:
x[ A ] =  0.0
y[ A ] =  0.0
x[ B ] =  0.0
y[ B ] =  0.0
x[ C ] =  1.0
y[ C ] =  0.0
x[ D ] =  1.0
y[ D ] =  0.0
x[ E ] =  2.0
y[ E ] =  0.0
x[ F ] =  3.0
y[ F ] =  0.0
x[ G ] =  3.0
y[ G ] =  0.0
x[ H ] =  4.0
y[ H ] =  0.0
x[ I ] =  6.0
y[ I ] =  0.0
z =  0.0


In [76]:
problem_15b = ConcreteModel()

# define set L 1..24
problem_15b.L = RangeSet(1,24)

# define set B 1..22
problem_15b.B = RangeSet(1,22)

# population dictionary
population = {1: 65333, 2: 103022, 3: 44088, 4: 56933, 5: 100358,
        6: 166906, 7: 78097, 8: 96991, 9: 47830, 10: 103087,
        11: 98172, 12: 67638, 13: 169659, 14: 151544, 15: 126496,
        16: 94383, 17: 103975, 18: 100276, 19: 98257, 20: 65440,
        21: 92170, 22: 8971}

# define param P{B}
problem_15b.P = Param(problem_15b.B, initialize=population)

# coverage dictionary
coverage_matrix = {
    (1, 1): 1, (1, 2): 1,
    (2, 1): 1, (2, 19): 1,
    (3, 2): 1, (3, 3): 1, (3, 19): 1,
    (4, 19): 1, (4, 20): 1,
    (5, 10): 1, (5, 17): 1, (5, 19): 1,
    (6, 18): 1, (6, 19): 1,
    (7, 17): 1, (7, 18): 1, (7, 22): 1,
    (8, 17): 1, (8, 22): 1,
    (9, 15): 1, (9, 16): 1, (9, 17): 1,
    (10, 10): 1, (10, 11): 1, (10, 16): 1, (10, 17): 1,
    (11, 13): 1, (11, 15): 1, (11, 16): 1,
    (12, 14): 1, (12, 15): 1, (12, 21): 1,
    (13, 14): 1, (13, 21): 1,
    (14, 12): 1, (14, 13): 1,
    (15, 7): 1, (15, 13): 1, (15, 14): 1, (15, 21): 1,
    (16, 7): 1, (16, 21): 1,
    (17, 8): 1, (17, 9): 1, (17, 10): 1, (17, 11): 1,
    (18, 3): 1, (18, 4): 1, (18, 8): 1, (18, 9): 1,
    (19, 2): 1, (19, 3): 1, (19, 4): 1,
    (20, 4): 1, (20, 5): 1,
    (21, 7): 1, (21, 8): 1,
    (22, 5): 1, (22, 6): 1, (22, 7): 1,
    (23, 5): 1, (23, 6): 1,
    (24, 2): 1, (24, 4): 1, (24, 6): 1,
}

coverage_dict = {}
for i in range(1, 25):
    for j in range(1, 23):
        if (i, j) in coverage_matrix:
            coverage_dict[(i, j)] = 1
        else:
            coverage_dict[(i, j)] = 0

print(coverage_dict)

# define param A{L,B}
problem_15b.A = Param(problem_15b.L, problem_15b.B, initialize=coverage_dict)

# var x binary for each element in L
problem_15b.x = Var(problem_15b.L, domain=Binary)

# define objective function sum{l in L, b in B} P[b] * A[l,b] * x[l]
problem_15b.z = Objective(expr=sum(problem_15b.P[b] * problem_15b.A[l, b] * problem_15b.x[l] for l in problem_15b.L for b in problem_15b.B), sense=maximize)

# define constraint sum l in L x[l] <= 4
problem_15b.r1 = Constraint(expr=sum(problem_15b.x[l] for l in problem_15b.L) <= 4)

# solve
solver = SolverFactory('gurobi')
solver.solve(problem_15b)

print('Solución optima:')
print('x[l] = ')
for l in problem_15b.L:
        print('x[', l, '] = ', value(problem_15b.x[l]))
print('z = ', value(problem_15b.z))


{(1, 1): 1, (1, 2): 1, (1, 3): 0, (1, 4): 0, (1, 5): 0, (1, 6): 0, (1, 7): 0, (1, 8): 0, (1, 9): 0, (1, 10): 0, (1, 11): 0, (1, 12): 0, (1, 13): 0, (1, 14): 0, (1, 15): 0, (1, 16): 0, (1, 17): 0, (1, 18): 0, (1, 19): 0, (1, 20): 0, (1, 21): 0, (1, 22): 0, (2, 1): 1, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (2, 6): 0, (2, 7): 0, (2, 8): 0, (2, 9): 0, (2, 10): 0, (2, 11): 0, (2, 12): 0, (2, 13): 0, (2, 14): 0, (2, 15): 0, (2, 16): 0, (2, 17): 0, (2, 18): 0, (2, 19): 1, (2, 20): 0, (2, 21): 0, (2, 22): 0, (3, 1): 0, (3, 2): 1, (3, 3): 1, (3, 4): 0, (3, 5): 0, (3, 6): 0, (3, 7): 0, (3, 8): 0, (3, 9): 0, (3, 10): 0, (3, 11): 0, (3, 12): 0, (3, 13): 0, (3, 14): 0, (3, 15): 0, (3, 16): 0, (3, 17): 0, (3, 18): 0, (3, 19): 1, (3, 20): 0, (3, 21): 0, (3, 22): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0, (4, 5): 0, (4, 6): 0, (4, 7): 0, (4, 8): 0, (4, 9): 0, (4, 10): 0, (4, 11): 0, (4, 12): 0, (4, 13): 0, (4, 14): 0, (4, 15): 0, (4, 16): 0, (4, 17): 0, (4, 18): 0, (4, 19): 1, (4, 20): 1, (4,

Solución optima:
x[l] = 
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0
x[ 5 ] =  0.0
x[ 6 ] =  0.0
x[ 7 ] =  0.0
x[ 8 ] =  0.0
x[ 9 ] =  0.0
x[ 10 ] =  1.0
x[ 11 ] =  1.0
x[ 12 ] =  1.0
x[ 13 ] =  0.0
x[ 14 ] =  0.0
x[ 15 ] =  1.0
x[ 16 ] =  0.0
x[ 17 ] =  0.0
x[ 18 ] =  0.0
x[ 19 ] =  0.0
x[ 20 ] =  0.0
x[ 21 ] =  0.0
x[ 22 ] =  0.0
x[ 23 ] =  0.0
x[ 24 ] =  0.0
z =  1651835.0
